In [1]:
import os
import sys

import numpy as np
import pandas as pd
import scipy.stats
import itertools

%load_ext watermark


If you need to install eqtk, you can uncomment and run the cell below. Or check out https://eqtk.github.io/getting_started/eqtk_installation.html. It can take several seconds to load the package into python. Additionally, the first time you run the equilibrium solver (eqtk.solve) may take longer than subsequent calls as the program compiles the code with Numba


In [2]:
# !{sys.executable} -m pip install --upgrade eqtk

In [3]:
import eqtk

In [4]:
#load some utility functions
pwd = os.path.abspath('../')
sys.path.append(os.path.join(pwd, 'code/')) 
from utilities import *

Simulate single layer dimerization network with a single input monomer (i.e. $x \in R$). 
Use latin hypercube sampling (via https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.LatinHypercube.html) to draw $K$ values from the range ($10^{-6}$, $10^6$) and accessory monomer levels ($a$) from the range ($10^{-3}$, $10^3$).

Additionally, for each sampled parameter set, iterate through each parameter, titrating it's value while hollding the remainig parameters fixed. 


In [5]:
def sample_params(m, k, lb, ub, centered = True, seed = 42):
    """
    For network of size m, sample k parameter sets (each parameter set includes Kij values and accessory monomer levels) 
    from log uniform latin hypercube with log10(lower bounds) = lb and log10(upper bounds) = ub. 
    """
    num_rxns = number_of_dimers(m)
    num_params = num_rxns + m-1 #assuming 1-input monomer
    lhs_sampler =  scipy.stats.qmc.LatinHypercube(d=num_params, centered=centered, seed=seed)
    param_sets = lhs_sampler.random(n=k)
    param_sets = scipy.stats.qmc.scale(param_sets, lb, ub)
    return np.power(10, param_sets)


In [10]:
def run_eqtk(N, C0, params, num_rxns, acc_monomer_ind):
    """
    Run eqtk.solve given the input stoichiometry matrix (N), initial concentrations (C0), and parameters (params)
    Parameters includes includes Kij values and accessory monomer levels. 
    """
    K = params[:num_rxns]
    C0[:,acc_monomer_ind] = params[num_rxns:]

    return eqtk.solve(c0=C0, N=N, K=K)



In [13]:
def simulate_networks(m, t = 10, k = 100, n = 5, input_lb = -3, input_ub = 3, 
                      param_sets = None, param_lb = None, param_ub = None,
                      centered = True, seed = 42, titrate_params = True, 
                      save = False, outfileprefix = ''):
    """
    Run simulations for dimer networks of size m and input titration size t 
    with k different parameter universes. 
    
    Parameters
    ----------
    m : int. 
        Number of monomer species. 
    t : int. Default 10. 
        Number of values to titrate the input monomer species. 
        Values spaced evenly on a log10 scale
    k : int. Default 100
        Number of parameter sets to sample. 
    n : int. Default 5. 
        If titrate_params = True, n indicates the number of values to titrate each parameter
    input_lb : int. Default -3
        lower bound for titrating the input monomer species. log10 scale
    input_ub : int. Default 3
        upper bound for titrating the input monomer species. log10 scale
    param_sets : None or array_like shape (k, num_parameters). Default None. 
        Parameters sets for simulating dimerization networks. 
        If None, sample_params to draw parameters from latin-hypercube. 
    param_lb : array_like shape (num_parameters,) 
        Lower bounds for sampling network parameters. 
    param_ub : array_like shape (num_parameters,) 
        Upper bounds for sampling network parameters
    centered : bool. Default True. 
        Indicates whether to center the draws from the latin-hypercube sampler
    seed : int. Default 42. 
        seed for LHS sampler
    titrate_params : Bool. Default True. 
        If true, then for each parameter universe, iterate through each parameter,
        titrating it's value while hollding the remainig parameters fixed.  
    save : Bool. Default False
        If True, save the parameter sets, initial concentration array (C0) and equlibrium concentration array (S_all)
    outfileprefix : string. Default ''
        relative path for saving figure.     
    Returns
    -------
    param_sets : array_like shape (k, num_parameters)
        Each parameter set (row) contains the bindnig affinities (Kij) and 
        accessory monomer concentrations (a) use to solve for equilibrium concentrations of 
        monomers & dimers. 
    C0 : array_like shape (t, number of species)
        Initial concentration array used for eqtk.solve
    S_all : array_like 
        Equlibrium concentration of all species (monomers & dimers)
        for all parameter sets. 
        If titrate_params = True then shape (t, num_species, k, num_params, n)
        If titrate_params = False then shape then shape (t, num_species, k)
        
    """
    #Create stoichiometry matrix and initial concnetration matrix 
    N = make_nXn_stoich_matrix(m)
    num_rxns = N.shape[0]
    M0_min = [input_lb] + [0] * (m-1)
    M0_max = [input_ub] + [0] * (m-1)
    num_conc = [t] + [1] * (m-1)

    C0 = make_C0_grid(m, M0_min=M0_min, M0_max=M0_max, num_conc=num_conc)
    
    num_params = num_rxns + (m-1)
        
    acc_monomer_ind = np.arange(1,m)
    
    #Parameter bounds for sampling and for titration
    if param_lb is None:
        param_lb = [-6]*num_rxns + [-3]*(m-1)

    if param_ub is None:
        param_ub = [6]*num_rxns + [3]*(m-1)
            
    if param_sets is None:
        #Generate random parameter sets
        param_sets = sample_params(m, k, param_lb, param_ub, centered=centered, seed=seed)
    
    if titrate_params:
        S_all = np.zeros((C0.shape[0], C0.shape[1], k, num_params, n))
        for pset_index in range(k):
            for param_index in range(num_params):
                pset_new = np.tile(param_sets[pset_index], (n, 1))
                #Titrate param
                param_titration = np.logspace(param_lb[param_index], param_ub[param_index], n)
                pset_new[:,param_index] = param_titration

                for titration_index in range(n):
                    S_all[:,:,pset_index,param_index,titration_index] = run_eqtk(N, C0.copy(), pset_new[titration_index], num_rxns, acc_monomer_ind)
        if save:
            np.save(f'{outfileprefix}/param_sets_{m}M_{k}k_paramTitration.npy', param_sets)
            np.save(f'{outfileprefix}/C0_{m}M_{k}k_paramTitration.npy', C0)
            np.save(f'{outfileprefix}S_all_{m}M_{k}k_paramTitration.npy', S_all)
    else:
        S_all = np.zeros((C0.shape[0], C0.shape[1], k))
        for pset_index, pset in enumerate(param_sets):
            S_all[:,:,pset_index] = run_eqtk(N, C0.copy(), pset, num_rxns, acc_monomer_ind)
        if save:
            np.save(f'{outfileprefix}param_sets_{m}M_{k}k.npy', param_sets)
            np.save(f'{outfileprefix}C0_{m}M_{k}k.npy', C0)
            np.save(f'{outfileprefix}S_all_{m}M_{k}k.npy', S_all)
    return param_sets, C0, S_all
                

Run simulations for networks with 3-6 monomer species. For each network size, sample 100 different initial parameter sets and titrate each parameter through 5 log-spaced levels. 

In [6]:
outfileprefix = '../data/20220420_1input_randomParams/'
if not os.path.isdir(outfileprefix):
    os.mkdir(outfileprefix)
    

In [16]:
param_sets_3M, C0_3M, S_all_3M = simulate_networks(m = 3, t = 10, k = 100, n = 5, 
                                                   input_lb = -3, input_ub = 3,
                                                   centered = True, seed = 42, save=True, 
                                                   outfileprefix=outfileprefix)


In [18]:
#4 monomer network
_, _, _ = simulate_networks(m = 4, t = 10, k = 100, n = 5, 
                            input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, save=True, 
                            outfileprefix=outfileprefix)


In [19]:
#5 monomer network
_, _, _ = simulate_networks(m = 5, t = 10, k = 100, n = 5, 
                            input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, save=True, 
                            outfileprefix=outfileprefix)


In [20]:
#6 monomer network
_, _, _ = simulate_networks(m = 6, t = 10, k = 100, n = 5, 
                            input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, save=True,
                            outfileprefix=outfileprefix)


Also simulate 1000 random parameter sets for different sized networks without titrating any of the parameters. 

In [21]:
_, _, _ = simulate_networks(m = 3, k = 1000, input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, titrate_params = False,
                            save=True, outfileprefix=outfileprefix)


In [22]:
_, _, _ = simulate_networks(m = 4, k = 1000, input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, titrate_params = False,
                            save=True, outfileprefix=outfileprefix)


In [23]:
_, _, _ = simulate_networks(m = 5, k = 1000, input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, titrate_params = False,
                            save=True, outfileprefix=outfileprefix)

In [24]:
_, _, _ = simulate_networks(m = 6, k = 1000, input_lb = -3, input_ub = 3, 
                            centered = True, seed = 42, titrate_params = False,
                            save=True, outfileprefix=outfileprefix)

Move to 20220420_plot1inputTitration_randomParams.ipynb notebook for plotting

In [7]:
%watermark --iversions


pandas: 1.4.1
numpy : 1.20.3
sys   : 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
scipy : 1.7.3
eqtk  : 0.1.2

